# Setup of Spark Environment

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

# Workflow using SparkSQL and Dataframes

Start with T as a dataframe.

T1: Filter out (drop) the transactions from T whose total amount is less than 200

T2: Over T1, group the transactions by the Number of Items it has, and for each group calculate the sum of total amounts, the average of total amounts, the min and the max of the total amounts.

Report back T2 to the client side.

T3: Over T1, group the transactions by customer ID, and for each group report the customer ID, and the transactions’ count.

T4: Filter out (drop) the transactions from T whose total amount is less than 600.

T5: Over T4, group the transactions by customer ID, and for each group report the customer ID, and the transactions’ count.

T6: Select the customer IDs whose T5.count * 5 < T3.count.

Report back T6 to the client side.

In [43]:
# Full Code

# Imports & setup
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark
from pyspark.sql.functions import col, sum, avg, min, max, count

# Import transaction data
transactions_df = spark.read.csv("Transactions.txt", header=False, inferSchema=True)
# Add column headers
transaction_column_headers = ["TransID", "CustID", "TransTotal", "TransNumItems", "TransDesc"]
for i, col_name in enumerate(transaction_column_headers):
    transactions_df = transactions_df.withColumnRenamed("_c" + str(i), col_name)
# transactions_df.printSchema()
# transactions_df.show()

# Start with T (transactions) as a dataframe
T = transactions_df
# print("Original T:")
# T.show()

# T1: Filter out (drop) the transactions from T whose total amount is less than $200
T1 = T.filter(T.TransTotal >= 200)
# print("T1:")
# T1.show()

# T2: Over T1, group the transactions by the Number of Items it has,
# and for each group calculate the sum of total amounts, the average of total amounts, the min and the max of the total amounts.
T2 = T1.groupBy('TransNumItems').agg(
    sum('TransTotal'),
    avg('TransTotal'),
    min('TransTotal'),
    max('TransTotal')
).orderBy('TransNumItems', ascending=True)
# Report back T2 to the client side
print("T2: sum, average, min, and max of total amounts of each group based on number of items with a total amount less than $200")
T2.show()

# T3: Over T1, group the transactions by customer ID
T3 = T1.groupBy('CustID').agg(count('CustID').alias('NumTransT3')).select('CustID','NumTransT3')
# For each group report the customer ID, and the transactions’ count
print("T3: customer ID and transactions’ count of each customer with a total amount less than $200")
T3.show()

# T4: Filter out (drop) the transactions from T whose total amount is less than $600
T4 = T.filter(T.TransTotal >= 600)
# print("T4:")
# T4.show()

# T5: Over T4, group the transactions by customer ID
T5 = T4.groupBy('CustID').agg(count('CustID').alias('NumTransT5')).select('CustID','NumTransT5')
# For each group report the customer ID, and the transactions’ count
print("T5: customer ID and transactions’ count of each customer with a total amount less than $600")
T5.show()

# T6: Select the customer IDs whose T5.count * 5 < T3.count
# Join T5 with T3 on 'CustID' to get access to 'NumTransT3'
T5_with_T3 = T5.join(T3, 'CustID', 'inner')
# T5_with_T3.show()
# Select the customer IDs whose T5.count * 5 < T3.count
T6 = T5_with_T3.filter((T5_with_T3['NumTransT5'] * 5) < T5_with_T3['NumTransT3']).select('CustID')
# Report back T6 to the client side
print("T6: customer IDs whose T5.count * 5 < T3.count")
T6.show()


T2: sum, average, min, and max of total amounts of each group based on number of items with a total amount less than $200
+-------------+--------------------+-----------------+---------------+---------------+
|TransNumItems|     sum(TransTotal)|  avg(TransTotal)|min(TransTotal)|max(TransTotal)|
+-------------+--------------------+-----------------+---------------+---------------+
|            1| 2.429269206697057E8|599.7529179960392|      200.00142|       999.9957|
|            2|2.4257767184206057E8|600.2852529102918|      200.00055|      999.99884|
|            3|2.4293303848863894E8| 600.156228123807|       200.0017|      999.99884|
|            4|2.4195768084812087E8|599.8970596137199|          200.0|      999.99805|
|            5|2.4224993707820743E8| 599.837906492631|      200.00012|       999.9995|
|            6|2.4252875229645818E8|600.1047950642795|      200.00104|      999.99927|
|            7|2.4212435738319233E8|600.6662417436977|      200.00023|      999.99945|
|       